# Data Validation

For more details and information about data validation, see the documentation of [lamindb](https://lamin.ai/docs/db/guide/data-validation).

In [ ]:
import nbproject as nb
from lnschema_core import DObject, Pipeline, Run, Storage, Usage
from lndb_setup import settings
from sqlmodel import Session, select
from pydantic import ValidationError

import pytest

nb.header()

In [ ]:
# non-optional created_at field is correctly ignored (server-side autopopulation)
pipeline = Pipeline()

In [ ]:
# invalid type for created_at field is correctly raised
with pytest.raises(ValidationError):
    pipeline = Pipeline(created_at="invalid type")

In [ ]:
# complex type validation works correctly (no error raised for valid input)
run = Run(name="Test Run", pipeline=pipeline)

In [ ]:
# complex type validation works correctly (ORM functionality is maintained)
assert run.pipeline

In [ ]:
# complex type validation works correctly (strict type checking for relationship fields)
with pytest.raises(TypeError):
    run = Run(name="Test Run", pipeline="invalid pipeline")

In [ ]:
# dobject validation works correctly (missing field errors are raised even when dobject is instantiated with custom constructor)
with pytest.raises(ValidationError):
    dobject = DObject(name="no data objects")

In [ ]:
# dobject validation works correctly (no errors raised when dobject is conformantly instantiated with custom constructor)
with Session(settings.instance.engine) as session:
    results = session.exec(select(Storage)).all()
    for store in results:
        if store.root == settings.instance.storage.root.as_posix():
            storage = store

dobject = DObject(name="mydobject", run_id=nb.meta.store.id, storage_id=storage.id)

In [ ]:
# enumeration validation works correctly (no errors raised when a valid enumeration member is passed)
assert Usage(type="ingest", dobject_id=dobject.id)

In [ ]:
# enumeration validation works correctly (error raised when invalid enumeration member is passed)
with pytest.raises(ValidationError):
    Usage(type="invalid member", dobject_id=dobject.id)